In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def compute_program_memory():
    import os
    import psutil
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1e9
def print_memory():
    print(f"Memory usage: {compute_program_memory()} GB")
print_memory()

Memory usage: 0.35606528 GB


In [3]:
from speedy_utils.all import *
from fastcore.all import *

setup_logger("D")

DEBUG    | utils_print.py:301 (setup_logger) - Logging set to DEBUG


In [14]:
import cloudpickle
from multiprocessing import Pool
import functools

# A nested function we want to parallelize
def w():
    def f(x):
        return x + 1
    inputs = range(100)
    return f, inputs

def _process_task(task):
    # This function is just a pass-through for demonstration
    func, x = task
    return func(x)

if __name__ == "__main__":
    # Grab our function and inputs
    func, inputs = w()

    # Serialize the function using cloudpickle
    pickled_f = cloudpickle.dumps(func)
    
    # We'll wrap the deserialization logic in a small wrapper
    def wrapper(x):
        f_unpickled = cloudpickle.loads(pickled_f)
        return f_unpickled(x)

    # Now we can map over the inputs with a standard multiprocessing.Pool,
    # because each call to `wrapper(x)` re-unpickles `func` internally.
    with Pool(4) as p:
        results = p.map(wrapper, inputs)

    print(results[:10])  # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [9]:
print_memory()

Memory usage: 0.21159936 GB


In [8]:
%%time
from fastcore.parallel import parallel

# Test with multiple inputs
input_values = [5_000, 100_00, 15_00, 20_00]  # Finding different prime numbers
results = parallel(f, input_values, n_workers=4)  # Use 4 processes
print(results)


[5001, 10001, 1501, 2001]
CPU times: user 6.17 ms, sys: 36.3 ms, total: 42.4 ms
Wall time: 54.5 ms


In [17]:
%%time
results = parallel(f, input_values, n_workers=4, threadpool=True)
print(results)



[48611, 104729, 12553, 17389]
CPU times: user 4.87 s, sys: 12 ms, total: 4.88 s
Wall time: 4.88 s
